In [1]:
from encoder_layer import Encoder_block
from positional_encoding import Positional_Encoding
from datasets import load_from_disk
from transformers import BertTokenizerFast
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from torch.nn.utils.rnn import pad_sequence

torch.Size([1, 3, 512])


In [2]:
dataset = load_from_disk("kp20k_local")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'doc_bio_tags', 'extractive_keyphrases', 'abstractive_keyphrases', 'other_metadata'],
        num_rows: 530809
    })
    test: Dataset({
        features: ['id', 'document', 'doc_bio_tags', 'extractive_keyphrases', 'abstractive_keyphrases', 'other_metadata'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'document', 'doc_bio_tags', 'extractive_keyphrases', 'abstractive_keyphrases', 'other_metadata'],
        num_rows: 20000
    })
})

In [4]:
def tokenize_batch(batch):
    texts = [" ".join(tokens) for tokens in batch["document"]]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding=False,          # padding baad mein collator karega
        max_length=256,
        return_offsets_mapping=False
    )
    tokenized["word_ids"] = [tokenized.word_ids(i) for i in range(len(texts))]
    return tokenized

In [5]:
tokenized_dataset = dataset.map(tokenize_batch, batched=True, remove_columns=["document"])

In [6]:
tag2id = {"O": 0, "B": 1, "I": 2}

def align_labels(batch):
    labels = []
    for tags, word_ids in zip(batch["doc_bio_tags"], batch["word_ids"]):
        tags = [tag2id[t] for t in tags]
        new_labels = []
        prev_wid = None
        for wid in word_ids:
            if wid is None:
                new_labels.append(-100)
            elif wid != prev_wid and wid < len(tags):
                new_labels.append(tags[wid] if tags[wid] == 0 else 2)  # B ko I bana do subwords ke liye
            elif wid < len(tags) and tags[wid] in [1,2]:
                new_labels.append(2)
            else:
                new_labels.append(-100)
            prev_wid = wid
        labels.append(new_labels)
    batch["labels"] = labels
    return batch

tokenized_dataset = tokenized_dataset.map(align_labels, batched=True)

In [7]:
final_columns = ["input_ids", "attention_mask", "token_type_ids", "labels"]
tokenized_dataset = tokenized_dataset.remove_columns(
    [col for col in tokenized_dataset["train"].column_names if col not in final_columns]
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import DistilBertModel
tokenizer = BertTokenizerFast.from_pretrained("distilbert-base-uncased")
bert_emb = DistilBertModel.from_pretrained("distilbert-base-uncased").get_input_embeddings()
bert_emb.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Embedding(30522, 768, padding_idx=0)

In [10]:
projection = nn.Linear(768, 512).to(device)

In [11]:
with torch.no_grad():
    nn.init.xavier_uniform_(projection.weight)
    projection.bias.fill_(0)

In [12]:
d_model = 512
num_heads = 8
d_ff = 2048

In [13]:
# print("x:", x.shape)
# print("attention_mask:", attention_mask.shape)

In [14]:
encoder = Encoder_block(d_model=d_model, num_heads=num_heads, d_ff=d_ff).to(device)

In [15]:
classifier = nn.Linear(d_model, 3).to(device)

In [16]:
bert_emb.requires_grad = False

In [17]:
# optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(classifier.parameters()), lr=1e-4)
optimizer = torch.optim.AdamW(
    list(projection.parameters()) + list(encoder.parameters()) + list(classifier.parameters()),
    lr=1e-4
)
criterion = nn.CrossEntropyLoss(ignore_index=-100, weight=torch.tensor([0.05, 1.0, 1.0]).to(device))

In [18]:
def smart_collate(batch):
    input_ids = [torch.tensor(item["input_ids"]) for item in batch]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]

    # Pad kar do
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 ignore hota hai loss mein

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [29]:
from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=smart_collate
)

In [32]:
import logging
import os
from tqdm import tqdm

# Create a logs directory if it doesn't exist
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    filename='logs/training.log', 
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

num_epochs = 20

encoder.train()
classifier.train()

checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # Sirf embedding part BERT se
        with torch.no_grad():
            x = bert_emb(input_ids)  # (B, L, 768)

        x = projection(x)        

        # print("x.shape:", x.shape)  # should be (batch_size, seq_len, embed_dim)
        # print("attention_mask.shape:", attention_mask.shape)  # should be (batch_size, seq_len)

        # mask = attention_mask.unsqueeze(1).unsqueeze(2)   # (B, 1, 1, seq_len)
        encoder_out, attn_weights = encoder(x, mask=attention_mask)

        # Yaha se tera apna encoder shuru
        # encoder_out, attn_weights = encoder(x, mask=attention_mask)  # tera function mask support karta hoga na?
        logits = classifier(encoder_out)

        loss = criterion(logits.view(-1, 3), labels.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=total_loss / (loop.n + 1))

    avg_loss = total_loss / len(train_loader)
    
    logging.info(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
    loop.write(f"Epoch [{epoch+1}/{num_epochs}] completed. Average Loss: {avg_loss:.4f}")


    # Save checkpoint after each epoch
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pt")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': encoder.state_dict(),
        'classifier_state_dict': classifier.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
    }, checkpoint_path)
    
    logging.info(f"Checkpoint saved at {checkpoint_path}")

Epoch 1/20:   0%|▏                                                      | 67/16588 [00:07<30:57,  8.89it/s, loss=0.592]


KeyboardInterrupt: 